In [29]:
# import numpy as np
import os
from utils import embedding_model,chroma_client,DB_PATH,file_pattern
# from utils import gpt_model
from IPython.display import HTML, Markdown
import glob
import tiktoken
from pathlib import Path
import chromadb

In [30]:
import sys
sys.path.append(r"C:\Users\rauna\projects\llm_engineering\My Projects\Drug Chatbot")

In [31]:
COLLECTION_NAME = 'chunked_by_sentence_collection'

In [32]:
BASE_PATH = Path(r"C:\Users\rauna\projects\llm_engineering\My Projects\Drug Chatbot\Synthetic_files")
subfolders = ["Paracetamol", "Insulin"]


In [33]:
folder_paths = [ str(BASE_PATH/sf) for sf in subfolders]

In [34]:
file_patterns = []
for folder_path in folder_paths:
    file_patterns.append(os.path.join(folder_path, "*.txt"))

In [35]:
list_file_paths = []
for file_pattern in file_patterns : 
    list_file_paths.append(glob.glob(file_pattern))

file_paths = [item for sublist in list_file_paths for item in sublist]

In [36]:
for file_path in file_paths:
    filename = os.path.basename(file_path)
    print(filename)

Paracetamol_file_01.txt
Paracetamol_file_02.txt
Paracetamol_file_03.txt
Paracetamol_file_04.txt
Paracetamol_file_05.txt
Paracetamol_file_06.txt
Paracetamol_file_07.txt
Paracetamol_file_08.txt
Paracetamol_file_09.txt
Paracetamol_file_10.txt
Paracetamol_file_11.txt
Paracetamol_file_12.txt
Paracetamol_file_13.txt
Paracetamol_file_14.txt
Paracetamol_file_15.txt
Paracetamol_file_16.txt
Paracetamol_file_17.txt
Paracetamol_file_18.txt
Paracetamol_file_19.txt
Insulin_file_20.txt
Insulin_file_21.txt
Insulin_file_22.txt
Insulin_file_23.txt
Insulin_file_24.txt
Insulin_file_25.txt
Insulin_file_26.txt
Insulin_file_27.txt
Insulin_file_28.txt
Insulin_file_29.txt
Insulin_file_30.txt
Insulin_file_31.txt
Insulin_file_32.txt
Insulin_file_33.txt
Insulin_file_34.txt
Insulin_file_35.txt
Insulin_file_36.txt
Insulin_file_37.txt
Insulin_file_38.txt


In [ ]:
texts = []
filenames = []

In [ ]:
##READING FILES FROM FILE STORAGE PATH
for file_path in file_paths :
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read().strip()
            if content:  
                texts.append(content)
                filenames.append(os.path.basename(file_path))
                print(f"Loaded: {os.path.basename(file_path)}")
    except Exception as e:
        print(f"Error reading {file_path}: {e}")


In [ ]:
# How many characters in all the documents?

# files = glob.glob(file_paths, recursive=True)
print(f"Found {len(file_paths)} files in the knowledge base")

entire_knowledge_base = ""

for file_path in file_paths:
    with open(file_path, 'r', encoding='utf-8') as f:
        entire_knowledge_base += f.read()
        entire_knowledge_base += "\n\n"

print(f"Total characters in knowledge base: {len(entire_knowledge_base):,}")

In [37]:
gpt_model = 'gpt-4o-2024-11-20'

In [ ]:
# How many tokens in all the documents?

encoding = tiktoken.encoding_for_model(gpt_model)
tokens = encoding.encode(entire_knowledge_base)
token_count = len(tokens)
print(f"Total tokens for {MODEL}: {token_count:,}")

In [ ]:
file_data = []
for file_path in file_paths:
    with open(file_path, 'r', encoding = 'utf-8') as f:
        content = f.read().strip()
        file_data.append(content)

In [ ]:
# Average character lenght and token count per file

char_lengths = []
char_sum = 0
tokens_list = []
token_sum = 0

for a in file_data:
    char_lengths.append(len(a))
    tokens_list.append(len(encoding.encode(a)))

for length in char_lengths:
    char_sum += length

for token in tokens_list:
    token_sum += token

print(f'Average char length per file : {round(char_sum/len(char_lengths))}')
print(f'Average token count per file : {round(token_sum/len(tokens_list))}')

In [ ]:
def chunk_by_lines_with_overlap(file_paths, chunk_size, overlap_ratio=0.3):
    chunks = []
    fn = []
    pn = []
    for file_path in file_paths:
        filename = os.path.basename(file_path)

        with open(file_path, "r", encoding="utf-8") as f:
            lines = f.readlines()

        overlap = int(chunk_size * overlap_ratio)
        step = chunk_size - overlap  # how many new lines we move forward each chunk

        for i in range(0, len(lines), step):
            chunk = lines[i : i + chunk_size]
            if chunk:
                chunk_with_name = [f"Filename: {filename}\n"] + chunk
                chunks.append(chunk_with_name)
                fn.append({'file_name' : filename})
                pn.append({'person' : filename.split('_')[0]})
                

            if i + chunk_size >= len(lines):
                break
    # print(fn)
    # print(pn)
    metadatas = [{**fil_nm,**per_nm} for fil_nm,per_nm in zip(fn,pn)]
    # print(metadatas)

    return chunks,metadatas

In [55]:
list_of_lists = chunk_by_lines_with_overlap(file_paths,8,0.3)

[{'file_name': 'Paracetamol_file_01.txt'}, {'file_name': 'Paracetamol_file_01.txt'}, {'file_name': 'Paracetamol_file_01.txt'}, {'file_name': 'Paracetamol_file_02.txt'}, {'file_name': 'Paracetamol_file_02.txt'}, {'file_name': 'Paracetamol_file_02.txt'}, {'file_name': 'Paracetamol_file_02.txt'}, {'file_name': 'Paracetamol_file_03.txt'}, {'file_name': 'Paracetamol_file_03.txt'}, {'file_name': 'Paracetamol_file_03.txt'}, {'file_name': 'Paracetamol_file_03.txt'}, {'file_name': 'Paracetamol_file_03.txt'}, {'file_name': 'Paracetamol_file_03.txt'}, {'file_name': 'Paracetamol_file_03.txt'}, {'file_name': 'Paracetamol_file_03.txt'}, {'file_name': 'Paracetamol_file_03.txt'}, {'file_name': 'Paracetamol_file_04.txt'}, {'file_name': 'Paracetamol_file_04.txt'}, {'file_name': 'Paracetamol_file_04.txt'}, {'file_name': 'Paracetamol_file_04.txt'}, {'file_name': 'Paracetamol_file_04.txt'}, {'file_name': 'Paracetamol_file_05.txt'}, {'file_name': 'Paracetamol_file_05.txt'}, {'file_name': 'Paracetamol_file_0

In [13]:
ids = [f'chunk_{i}' for i in range(len(list_of_lists[0]))] 

In [14]:
chunks = ["".join(chunk) for chunk in list_of_lists[0]]
metadatas = list_of_lists[1]

In [56]:
display(len(chunks))
display(len(metadatas))
display(len(ids))

212

212

212

In [15]:
sentence_embeddings = embedding_model.encode(chunks)

In [19]:
chroma_client = chromadb.PersistentClient(path=DB_PATH)

In [ ]:
collection = chroma_client.get_or_create_collection(
    name=COLLECTION_NAME,
    metadata={"description": "Sentence method embeddings using all-MiniLM-L6-v2"}
)

In [ ]:
collection.add(
    embeddings=sentence_embeddings,
    documents=chunks,
    metadatas=metadatas,
    ids=ids
)

In [ ]:
print(f"\n{'='*60}")
print(f"Model: all-MiniLM-L6-v2")
print(f"Embedding dimension: {sentence_embeddings.shape[1]}")
print(f"Number of files embedded: {sentence_embeddings.shape[0]}")
print(f"{'='*60}\n")